# Segmenting and Clustering Neighborhoods in Toronto - Part 3

#### In this assignment, I am segmenting and clustering neighborhoods in Toronto and joining the co-ordinates data from an another data source. Exploring and clustring the neighborhoods in Toronto.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

In [2]:
# Wikipedia URL
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
table_html = pd.read_html(URL)

In [4]:
print(f'Total tables: {len(table_html)}')

Total tables: 3


#### After the observation, I found that the first table is relevant to the question. Hence, I am using the fist table for the further work.

In [5]:
df_all = table_html[0]
df_all.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df_all.shape

(180, 3)

#### In the next dataframe, I am selecting the assigned Borough data.

In [7]:
df = df_all[df_all['Borough']!='Not assigned']
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 3)

#### The code given in the assignement to get co-ordinates was taking forever. Hence, I have used the CSV link to get the data of co-ordinates.

In [9]:
# Importing Geospactial data from CSV file
df_geospacial = pd.read_csv('https://cocl.us/Geospatial_data')
df_geospacial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### I have used the join method to get the co-ordinates to the main dataframe.

In [10]:
df = df.join(df_geospacial.set_index('Postal Code'), on='Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
df.shape

(103, 5)

In [12]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df_toronto['Borough'].unique()),df_toronto.shape[0]))

The dataframe has 4 boroughs and 39 neighborhoods.


In [14]:
# Installing required libraries
#! conda install -c conda-forge geopy --yes
#! conda install -c conda-forge folium=0.5.0 --yes

In [15]:
# Importing libraries required for geo graph
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium  # map rendering library

In [16]:
latitude = 43.806686
longitude = -79.194353
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.806686, -79.194353.


In [20]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto